# Segmenting and Clustering Neighbourhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

First, dependencies are downloaded

In [1]:
import requests
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import json
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported


# Part 1 
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

To create the above dataframe:
+ The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
+ Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
+ More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
+ If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
+ Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
+ In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### Create a dataframe with 3 columns: PostalCode, Borough and Neighborhood

In [8]:
#create dataframe with 3 columns containing information scraped from the Wikipedia page
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html = r.text

bs = BeautifulSoup(html)
table=bs.find('tbody')
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df = pd.DataFrame(data,columns=['PostalCodes','Borough','Neighbourhood'])
df.head()

,PostalCodes,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove cells with a Borough that is labelled 'Not Assigned'

In [9]:
# remove cells with a borough that is Not Assigned
df['Borough'].replace('Not assigned', np.nan,inplace=True)
df.dropna(subset=['Borough'],inplace=True)
df.head()

,PostalCodes,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Combine rows of neighborhoods with the same postal code

In [10]:
df2 = df.groupby(['PostalCodes','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df2.columns = ['PostalCodes','Borough','Neighbourhood']
df2.head()

,PostalCodes,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Postal codes with a borough but a not assigned neighborhood, change neighborhood to the same as borough

In [11]:
for index, row in df2.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head()

,PostalCodes,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Use the .shape method to print number of rows of the dataframe

In [12]:
df2.shape

(103, 3)

# Part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Despite multiple attempts, geocoder ran for hours and never completed, so the decision was made to utilise the csv file instead

In [13]:
import io
url2 = 'http://cocl.us/Geospatial_data'
g = requests.get(url2).content
s = pd.read_csv(io.StringIO(g.decode('utf-8')))
s.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
s.rename(columns={'Postal Code':'PostalCodes'},inplace=True)
s.sort_values(by = ['PostalCodes'],ascending=True)
s.head()

,PostalCodes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df2.sort_values(by=['PostalCodes'],ascending=True)
df2.head()

,PostalCodes,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df2 = pd.merge(df2,s,on='PostalCodes')
df2

,PostalCodes,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
+ to add enough Markdown cells to explain what you decided to do and to report any observations you make.
+ to generate maps to visualize your neighborhoods and how they cluster together. 

In [17]:
from geopy.geocoders import Nominatim
import json 
from pandas.io.json import json_normalize
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

#Foursquare credentials
CLIENT_ID = 'T1NIO3TDFZS402OHWBDJBFIRAY2Q0LZK1P3AM5HY1UOHXYUQ'
CLIENT_SECRET = 'HV2DQUDBZJXARPLC5RLWTZUXT1LJ52LK0FGBIBWIFRBNA11B'
VERSION = 20180605
LIMIT = 100 # to limit number of venues returned
radius = 500 #defines radius of search

### Functions defined

In [18]:
#function to extract the venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
#function to repear the same process to all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        #create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        #make GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #return only relevant info for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                            'Neighbourhood Latitude',
                            'Neighbourhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    return(nearby_venues)

In [20]:
#function to sort venues in descending order
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Explore neighbourhoods in Toronto

In [21]:
#explore venues in all neighbourhoods and create venue dataframe
toronto_venues = getNearbyVenues(names=df2['PostalCodes'], latitudes=df2['Latitude'],longitudes=df2['Longitude'])
print(toronto_venues.shape)

(2231, 7)


In [22]:
#find how many venues were returned for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 unique categories.


In [23]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1C,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Analyse neighbourhoods in Toronto

In [24]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")

#add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

#move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()
toronto_onehot.shape

(2231, 275)

In [25]:
#group rows by neighbourhoodand by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

(100, 275)

In [26]:
#create a dataframe of the 5 most common venues in each neighbourhood
num_top_venues = 10
indicators = ['st','nd','rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#create dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)
print(neighbourhoods_venues_sorted.shape)
neighbourhoods_venues_sorted

(100, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Moving Target,Bar,Yoga Studio,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Breakfast Spot,Rental Car Location,Pizza Place,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,M1G,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Lounge,Cupcake Shop,Curling Ice
5,M1J,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,M1K,Department Store,Hobby Shop,Coffee Shop,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,M1L,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Bakery,Park,Gift Shop,German Restaurant
8,M1M,American Restaurant,Motel,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
9,M1N,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


### Cluster Neighbourhoods by running k-means

In [27]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [28]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood
#add clustering labels
toronto_merged = df2
neighbourhoods_venues_sorted.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
toronto_merged.rename(columns={'PostalCodes':'Neighborhood'}, inplace=True)

#add clustering labels
neighbourhoods_venues_sorted.insert(0,"Cluster labels",kmeans.labels_)

#merge toronto_grouped with df2 to add latitude and longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')

toronto_merged.head()

,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Moving Target,Bar,Yoga Studio,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Breakfast Spot,Rental Car Location,Pizza Place,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Lounge,Cupcake Shop,Curling Ice


In [43]:
toronto_merged

,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Moving Target,Bar,Yoga Studio,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Breakfast Spot,Rental Car Location,Pizza Place,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Lounge,Cupcake Shop,Curling Ice
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Department Store,Hobby Shop,Coffee Shop,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0.0,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Bakery,Park,Gift Shop,German Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


In [45]:
#drop rows with cluster label as NaN
toronto_merged.dropna(subset=['Cluster labels'], inplace=True)
toronto_merged

,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Moving Target,Bar,Yoga Studio,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Breakfast Spot,Rental Car Location,Pizza Place,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Lounge,Cupcake Shop,Curling Ice
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Department Store,Hobby Shop,Coffee Shop,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0.0,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Bakery,Park,Gift Shop,German Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


Now to visualise

In [61]:
#create map
toronto_latitude = 43.7001
toronto_longitude = -79.4163
map_clusters = folium.Map(location=[toronto_latitude,toronto_longitude],zoom_start=11)

#set colour scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Examine Clusters

In [52]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Breakfast Spot,Rental Car Location,Pizza Place,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,Scarborough,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Caribbean Restaurant,Athletics & Sports,Lounge,Cupcake Shop,Curling Ice
5,Scarborough,0.0,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,Scarborough,0.0,Department Store,Hobby Shop,Coffee Shop,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,Scarborough,0.0,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Bakery,Park,Gift Shop,German Restaurant
8,Scarborough,0.0,American Restaurant,Motel,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
9,Scarborough,0.0,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
10,Scarborough,0.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
11,Scarborough,0.0,Breakfast Spot,Bakery,Smoke Shop,Middle Eastern Restaurant,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [53]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Etobicoke,1.0,Park,River,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant


In [54]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [56]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3.0,Park,Playground,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
23,North York,3.0,Park,Bank,Convenience Store,Flower Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
30,North York,3.0,Park,Airport,Bus Stop,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
40,East York,3.0,Park,Pizza Place,Coffee Shop,Convenience Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
44,Central Toronto,3.0,Park,Photography Studio,Swim School,Bus Line,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
50,Downtown Toronto,3.0,Park,Playground,Building,Trail,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
64,Central Toronto,3.0,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
74,York,3.0,Park,Women's Store,Market,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner
79,North York,3.0,Park,Bakery,Construction & Landscaping,Basketball Court,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [58]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Moving Target,Bar,Yoga Studio,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
